In [84]:
import pandas as pd

In [85]:
global_confirmed_raw_data = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
global_death_raw_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
global_recovered_raw_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [86]:
global_confirmed_raw_data.head() 

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,38288,38304,38324,38398,38494,38520,38544,38572,38606,38641,38716,38772,38815,38855,38872,38883,38919,39044,39074,39096,39145,39170,39186,39192,39227,39233,39254,39268,39285,39290,39297,39341,39422,39486,39548,39616,39693,39703,39799,39870
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,9844,9967,10102,10255,10406,10553,10704,10860,11021,11185,11353,11520,11672,11816,11948,12073,12226,12385,12535,12666,12787,12921,13045,13153,13259,13391,13518,13649,13806,13965,14117,14266,14410,14568,14730,14899,15066,15231,15399,15570
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,45469,45773,46071,46364,46653,46938,47216,47488,47752,48007,48254,48496,48734,48966,49194,49413,49623,49826,50023,50214,50400,50579,50754,50914,51067,51213,51368,51530,51690,51847,51995,52136,52270,52399,52520,52658,52804,52940,53072,53325
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1199,1215,1215,1215,1261,1261,1301,1301,1344,1344,1344,1438,1438,1483,1483,1564,1564,1564,1681,1681,1753,1753,1836,1836,1836,1966,1966,2050,2050,2110,2110,2110,2370,2370,2568,2568,2696,2696,2696,2995
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2805,2876,2935,2965,2981,3033,3092,3217,3279,3335,3388,3439,3569,3675,3789,3848,3901,3991,4117,4236,4363,4475,4590,4672,4718,4797,4905,4972,5114,5211,5370,5402,5530,5725,5725,5958,6031,6246,6366,6488


In [87]:
print(global_confirmed_raw_data.shape, global_death_raw_data.shape, global_recovered_raw_data.shape)

(267, 269) (267, 269) (254, 269)


In [88]:
global_confirmed_raw_data['Country/Region'].value_counts()

China             33
Canada            14
France            11
United Kingdom    11
Australia          8
                  ..
Hungary            1
Guyana             1
Rwanda             1
Jordan             1
Kosovo             1
Name: Country/Region, Length: 189, dtype: int64

In [89]:
date = '10/8/20'

In [90]:
global_confirmed_raw_data_now = global_confirmed_raw_data[['Province/State', 'Country/Region', 'Lat', 'Long', date]]
global_death_raw_data_now = global_death_raw_data[['Province/State', 'Country/Region', 'Lat', 'Long', date]]
global_recovered_raw_data_now = global_recovered_raw_data[['Province/State', 'Country/Region', 'Lat', 'Long', date]]

In [91]:
global_confirmed_raw_data_now.head()

,Province/State,Country/Region,Lat,Long,10/8/20
0,NaN,Afghanistan,33.93911,67.709953,39616
1,NaN,Albania,41.15330,20.168300,14899
2,NaN,Algeria,28.03390,1.659600,52658
3,NaN,Andorra,42.50630,1.521800,2568
4,NaN,Angola,-11.20270,17.873900,5958


In [92]:
global_confirmed_raw_data_now.rename(columns={'10/8/20':'Confirmed'}, inplace=True)
global_death_raw_data_now.rename(columns={'10/8/20':'Death'}, inplace=True)
global_recovered_raw_data_now.rename(columns={'10/8/20':'Recovered'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [93]:
global_data_now = pd.merge(global_confirmed_raw_data_now,global_death_raw_data_now,on=['Province/State', 'Country/Region', 'Lat', 'Long'])
global_data_now = pd.merge(global_data_now,global_recovered_raw_data_now,on=['Province/State', 'Country/Region', 'Lat', 'Long'])

In [94]:
global_data_now

,Province/State,Country/Region,Lat,Long,Confirmed,Death,Recovered
0,NaN,Afghanistan,33.939110,67.709953,39616,1470,33058
1,NaN,Albania,41.153300,20.168300,14899,411,9215
2,NaN,Algeria,28.033900,1.659600,52658,1783,36958
3,NaN,Andorra,42.506300,1.521800,2568,54,1715
4,NaN,Angola,-11.202700,17.873900,5958,208,2635
...,...,...,...,...,...,...,...
213,NaN,Venezuela,6.423800,-66.589700,81019,678,72196
214,NaN,West Bank and Gaza,31.952200,35.233200,43256,359,36584
215,NaN,Western Sahara,24.215500,-12.885800,10,1,8
216,NaN,Zambia,-13.133897,27.849332,15301,335,14365


Import map interface

In [95]:
! pip install geopandas
! pip install folium

In [96]:
import numpy as np
import geopandas as gpd
from geopandas.tools import geocode
import math
from collections import namedtuple

import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster, TimestampedGeoJson

import datetime
import os

Create map

In [111]:
map_now = folium.Map(location=[30, 0], 
                 tiles = "CartoDB dark_matter",
                 detect_retina = True,
                 zoom_start=2,
                 no_touch=True)

In [112]:
world_countries_geo = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json"

folium.Choropleth(
    geo_data=world_countries_geo,
    name='choropleth',
#     data=filtered_data_last,
#     columns=['Province/State', 'Confirmed'],
    key_on='feature.properties.name',
    fill_color='green',
    fill_opacity=0.18,
    line_opacity=0.7
).add_to(map_now)

In [113]:
for i in range(len(global_data_now)):
  lat = global_data_now.loc[i, 'Lat']
  lon = global_data_now.loc[i, 'Long']
  country = global_data_now.loc[i, 'Country/Region']
  province = global_data_now.loc[i, 'Province/State']
  recovered = global_data_now.loc[i, 'Recovered']
  death = global_data_now.loc[i, 'Death']
  confirmed = global_data_now.loc[i, 'Confirmed']

  if confirmed-recovered-death <= 0 :
    _radius_rec = 0
  else:
    _radius_rec = math.log2(confirmed-recovered-death)

  _popup_rec = str(country) + ' ' + str(province) + '(Confirmed='+str(confirmed) + ' Deaths=' + str(death) + ' Recovered=' + str(recovered) + ')'

  _color_rec = '#81D8D0'

  folium.CircleMarker(location = [lat,lon], 
                        radius = _radius_rec, 
                        popup = _popup_rec, 
                        color = _color_rec, 
                        fill_opacity = 0.1,
                      weight = 2, 
                      fill = True, 
                      fillColor = _color_rec).add_to(map_now) 

In [114]:
map_now

### Reference

[2019-nCoV疫情传播可视化和预测--交互式地图分析
](https://www.kesci.com/home/project/5e44d8d15f2816002cec62a0)

[Visualizing Bike Mobility in London using Interactive Maps and Animations](https://towardsdatascience.com/visualizing-bike-mobility-in-london-using-interactive-maps-for-absolute-beginners-3b9f55ccb59)

[Tracking the spread of 2019 Coronavirus](https://www.kaggle.com/gpreda/tracking-the-spread-of-2019-coronavirus)